In [ ]:
## Milestone 3: Create the star schema of the database, ensuring that the columns are of the correct data type.

1. Change the data types to correspond to those seen in the table below.

    +------------------+--------------------+--------------------+
    |   orders_table   | current data type  | required data type |
    +------------------+--------------------+--------------------+
    | date_uuid        | TEXT               | UUID               |
    | user_uuid        | TEXT               | UUID               |
    | card_number      | TEXT               | VARCHAR(?)         |
    | store_code       | TEXT               | VARCHAR(?)         |
    | product_code     | TEXT               | VARCHAR(?)         |
    | product_quantity | BIGINT             | SMALLINT           |
    +------------------+--------------------+--------------------+
The ? in VARCHAR should be replaced with an integer representing the maximum length of the values in that column.

https://www.commandprompt.com/education/how-to-alter-column-type-in-postgresql/#:~:text=Here%20is%20a%20simple%20syntax,after%20the%20ALTER%20COLUMN%20command.

In [ ]:
-- find longest value for card_number --> 19
SELECT max(length(CAST(card_number AS VARCHAR))) FROM orders_table;

SELECT max(length(CAST(store_code AS VARCHAR))) FROM orders_table; -- 12

SELECT max(length(CAST(product_code AS VARCHAR))) FROM orders_table; -- 11

-- To cast uuid you need to explicitly cast as uuid 
-- https://varunver.wordpress.com/2020/07/07/postgres-change-column-from-type-text-to-uuid/
-- https://www.sqlservercentral.com/articles/the-cast-operator-in-postgresql

In [ ]:
ALTER TABLE orders_table
    ALTER COLUMN date_uuid TYPE UUID USING date_uuid::UUID,
    ALTER COLUMN user_uuid TYPE UUID USING user_uuid::UUID,
    ALTER COLUMN card_number TYPE VARCHAR(19),
    ALTER COLUMN store_code TYPE VARCHAR(12),
    ALTER COLUMN product_code TYPE VARCHAR(11),
    ALTER COLUMN product_quantity TYPE SMALLINT;


2. The column required to be changed in the users table are as follows:

    +----------------+--------------------+--------------------+
    | dim_user_table | current data type  | required data type |
    +----------------+--------------------+--------------------+
    | first_name     | TEXT               | VARCHAR(255)       |
    | last_name      | TEXT               | VARCHAR(255)       |
    | date_of_birth  | TEXT               | DATE               |
    | country_code   | TEXT               | VARCHAR(?)         |
    | user_uuid      | TEXT               | UUID               |
    | join_date      | TEXT               | DATE               |
    +----------------+--------------------+--------------------+

In [ ]:
SELECT max(length(country_code::VARCHAR)) FROM dim_users; 


In [ ]:
ALTER TABLE dim_users
    ALTER COLUMN first_name TYPE VARCHAR(255),
    ALTER COLUMN last_name TYPE VARCHAR(255),
    ALTER COLUMN date_of_birth TYPE DATE,
    ALTER COLUMN country_code TYPE VARCHAR(2),
    ALTER COLUMN user_uuid TYPE UUID USING user_uuid::UUID,
    ALTER COLUMN join_date TYPE DATE;

3. There are two latitude columns in the store details table. Using SQL, merge one of the columns into the other so you have one latitude column.
--> do we need to resurrect the lat column? No. It is garbage. I am presuming this means to merge Latitude and Longitude. I've not cast either as floats anymore in Pandas so we can perform the concatenation here.

Then set the data types for each column as shown below:

    +---------------------+-------------------+------------------------+
    | store_details_table | current data type |   required data type   |
    +---------------------+-------------------+------------------------+
    | longitude           | TEXT              | FLOAT                  |
    | locality            | TEXT              | VARCHAR(255)           |
    | store_code          | TEXT              | VARCHAR(?)             |
    | staff_numbers       | TEXT              | SMALLINT               |
    | opening_date        | TEXT              | DATE                   |
    | store_type          | TEXT              | VARCHAR(255) NULLABLE  |
    | latitude            | TEXT              | FLOAT                  |
    | country_code        | TEXT              | VARCHAR(?)             |
    | continent           | TEXT              | VARCHAR(255)           |
    +---------------------+-------------------+------------------------+
There is a row that represents the business's website change the location column values where they're null to N/A.

In [ ]:
SELECT max(length(store_code::VARCHAR)) FROM dim_stores_details; -- 12

In [ ]:
ALTER TABLE dim_stores_details
    ALTER COLUMN longitude TYPE FLOAT USING longitude::FLOAT,
    ALTER COLUMN locality TYPE VARCHAR(255),
    ALTER COLUMN store_code TYPE VARCHAR(12),
    ALTER COLUMN staff_numbers TYPE SMALLINT,
    ALTER COLUMN opening_date TYPE DATE USING opening_date::DATE,
    ALTER COLUMN store_type TYPE VARCHAR(255),
    ALTER COLUMN store_type DROP NOT NULL, -- i think this makes it nullable
    ALTER COLUMN latitude TYPE FLOAT USING latitude::FLOAT,
    ALTER COLUMN country_code TYPE VARCHAR(2),
    ALTER COLUMN continent TYPE VARCHAR(255);

In [ ]:
--There is a row that represents the business's website change the location column values where they're null to N/A.
UPDATE 
    dim_stores_details
SET 
    address = 'N/A', locality='N/A', country_code='N/A', --latitude='N/A', longitude='N/A'
WHERE 
    store_code ILIKE 'WEB-1388012W';

--There is a row that represents the business's website change the location column values where they're null to N/A.

 

In [ ]:
--SELECT * FROM dim_stores_details ORDER BY level_0;

4a. You will need to do some work on the products table before casting the data types correctly.

The product_price column has a £ character which you need to remove using SQL. (I already did this so going to try to replace something else just for the craic)


In [ ]:
-- UPDATE
--     dim_products
-- SET 
--     category = REPLACE(category, 'boys', 'toys');
    

4b. The team that handles the deliveries would like a new human-readable column added for the weight so they can quickly make decisions on delivery weights.
Add a new column weight_class which will contain human-readable values based on the weight range of the product.

    +--------------------------+-------------------+
    | weight_class VARCHAR(?)  | weight range(kg)  |
    +--------------------------+-------------------+
    | Light                    | < 2               |
    | Mid_Sized                | >= 2 - < 40       |
    | Heavy                    | >= 40 - < 140     |
    | Truck_Required           | => 140            |
    +----------------------------+-----------------+

In [ ]:
   
ALTER TABLE 
    dim_products 
ADD 
    weight_class VARCHAR(14);


UPDATE dim_products
SET weight_class = 
    CASE
            WHEN 
                weight < 2 THEN 'Light'
            WHEN    
                weight >= 2 AND weight < 40 THEN 'Mid_Sized'
            WHEN    
                weight >= 40 AND weight < 140 THEN 'Heavy'
            WHEN 
                weight > 120 THEN 'Long'
    END; 


In [ ]:
SELECT * FROM dim_products LIMIT 5;

5. 
After all the columns are created and cleaned, change the data types of the products table.

You will want to rename the removed column to still_available before changing its data type.


In [ ]:

UPDATE
    dim_products
SET 
    removed = REPLACE(removed, 'Still_avaliable', 'True');

UPDATE
    dim_products
SET 
    removed = REPLACE(removed, 'Removed', 'False');

In [ ]:
-- after we do the replacing, we change the name
ALTER TABLE dim_products
RENAME COLUMN "removed" TO still_available;

In [ ]:
SELECT removed FROM dim_products LIMIT 30;

In [ ]:
SELECT max(length("EAN"::VARCHAR)) FROM dim_products; --17
SELECT max(length(product_code::VARCHAR)) FROM dim_products; --11
SELECT max(length(weight_class::VARCHAR)) FROM dim_products; --9


Make the changes to the columns to cast them to the following data types:

    +-----------------+--------------------+--------------------+
    |  dim_products   | current data type  | required data type |
    +-----------------+--------------------+--------------------+
    | product_price   | TEXT               | FLOAT              |
    | weight          | TEXT               | FLOAT              |
    | EAN             | TEXT               | VARCHAR(?)         |
    | product_code    | TEXT               | VARCHAR(?)         |
    | date_added      | TEXT               | DATE               |
    | uuid            | TEXT               | UUID               |
    | still_available | TEXT               | BOOL               |
    | weight_class    | TEXT               | VARCHAR(?)         |
    +-----------------+--------------------+--------------------+

In [ ]:
ALTER TABLE dim_products
    ALTER COLUMN product_price TYPE FLOAT, 
    ALTER COLUMN weight TYPE FLOAT,
    ALTER COLUMN "EAN" TYPE VARCHAR(17),
    ALTER COLUMN product_code TYPE VARCHAR(11),
    ALTER COLUMN date_added TYPE DATE USING date_added::DATE,
    ALTER COLUMN uuid TYPE UUID USING uuid::UUID,
    ALTER COLUMN still_available TYPE BOOL USING still_available::BOOL, 
    ALTER COLUMN weight_class TYPE VARCHAR(9);
    

6. Now update the date table with the correct types:

    +-----------------+-------------------+--------------------+
    | dim_date_times  | current data type | required data type |
    +-----------------+-------------------+--------------------+
    | month           | TEXT              | VARCHAR(?)         |
    | year            | TEXT              | VARCHAR(?)         |
    | day             | TEXT              | VARCHAR(?)         |
    | time_period     | TEXT              | VARCHAR(?)         |
    | date_uuid       | TEXT              | UUID               |
    +-----------------+-------------------+--------------------+

I have already changed this to a unified date format, so I'm going to clean what I have:

    +-----------------+-------------------+--------------------+
    | dim_date_times  | current data type | required data type |
    +-----------------+-------------------+--------------------+
    | date_uuid       | TEXT              | UUID               |
    +-----------------+-------------------+--------------------+

I decided here to keep the Timestamp which my table has, as this can be used in the Milestone 4 task.

In [ ]:
ALTER TABLE dim_date_times
    -- ALTER COLUMN date TYPE DATE USING date::DATE, -- I think I want to keep these are Timestamp values as this allows me to complete the final project
    ALTER COLUMN date_uuid TYPE UUID USING date_uuid::UUID

7. Now we need to update the last table for the card details.

Make the associated changes after finding out what the lengths of each variable should be:

    +------------------------+-------------------+--------------------+
    |    dim_card_details    | current data type | required data type |
    +------------------------+-------------------+--------------------+
    | card_number            | TEXT              | VARCHAR(?)         |
    | expiry_date            | TEXT              | VARCHAR(?)         |
    | date_payment_confirmed | TEXT              | DATE               |
    +------------------------+-------------------+--------------------+

I've also changed this, so going to use expiry date as a date.

In [ ]:
SELECT max(length(card_number::VARCHAR)) FROM dim_card_details; -- 19
SELECT max(length(expiry_date::VARCHAR)) FROM dim_card_details;

In [ ]:
   ALTER TABLE dim_card_details
      ALTER COLUMN card_number TYPE VARCHAR(19),
      ALTER COLUMN expiry_date TYPE DATE USING expiry_date::DATE,
      ALTER COLUMN date_payment_confirmed TYPE DATE USING date_payment_confirmed::DATE

8. Now that the tables have the appropriate data types we can begin adding the primary keys to each of the tables prefixed with dim.

Each table will serve the orders_table which will be the single source of truth for our orders.

Check the column header of the orders_table you will see all but one of the columns exist in one of our tables prefixed with dim.

We need to update the columns in the dim tables with a primary key that matches the same column in the orders_table.

Using SQL, update the respective columns as primary key columns.

    **Orders table:**           **Other table:**
    date_uuid               dim_date_times
    user_uuid               dim_users
    card_number             dim_card_details 
    store_code              dim_stores_details
    product_code            dim_products
    product_quantity        --doesn't appear

So we need to change these columns as primary keys in their respective tables

In [ ]:
ALTER TABLE dim_date_times ADD PRIMARY KEY (date_uuid);
ALTER TABLE dim_users ADD PRIMARY KEY (user_uuid);
ALTER TABLE dim_card_details ADD PRIMARY KEY (card_number);
ALTER TABLE dim_stores_details ADD PRIMARY KEY (store_code);
ALTER TABLE dim_products ADD PRIMARY KEY (product_code);


9. With the primary keys created in the tables prefixed with dim we can now create the foreign keys in the orders_table to reference the primary keys in the other tables.

Use SQL to create those foreign key constraints that reference the primary keys of the other table.

This makes the star-based database schema complete.

In [ ]:
ALTER TABLE orders_table
    ADD CONSTRAINT fk_orders_table_dim_date_times
    FOREIGN KEY (date_uuid) 
    REFERENCES dim_date_times(date_uuid);

ALTER TABLE orders_table
    ADD CONSTRAINT fk_orders_table_dim_users
    FOREIGN KEY (user_uuid) 
    REFERENCES dim_users(user_uuid);

ALTER TABLE orders_table
    ADD CONSTRAINT fk_orders_table_dim_card_details
    FOREIGN KEY (card_number) 
    REFERENCES dim_card_details(card_number);

ALTER TABLE orders_table
    ADD CONSTRAINT fk_orders_table_dim_stores_details
    FOREIGN KEY (store_code)
    REFERENCES dim_stores_details(store_code);

ALTER TABLE orders_table
    ADD CONSTRAINT fk_orders_table_dim_products
    FOREIGN KEY (product_code)
    REFERENCES dim_products(product_code);

    -- I haven't put any further constraints in here yet.
    